In [6]:
import pandas as pd
import os

In [7]:
all_files = os.listdir("/Users/keithwhitson/Projekte/russian-troll-tweets/")
all_files = filter(lambda x: 'csv' in x, all_files)
all_files = list(all_files)

In [9]:
df = pd.concat(map(lambda file: pd.read_csv(file, low_memory=False), all_files))
df["external_author_id"] = df["external_author_id"].astype(str)
df['alt_external_id'] = df['alt_external_id'].astype(str)
df['tweet_id'] = df['tweet_id'].astype(str)

In [13]:
df.to_parquet('russian_tweets.parquet',compression="snappy",partition_cols=["author"])

In [14]:
df = pd.read_parquet('russian_tweets.parquet',engine='pyarrow')

In [17]:
df.describe()

,following,followers,updates,retweet,new_june_2018
count,2.946207e+06,2.946207e+06,2.946207e+06,2.946207e+06,2.946207e+06
mean,3.448728e+03,7.055265e+03,1.051469e+04,4.404246e-01,2.095691e-01
std,5.625644e+03,1.463594e+04,1.770555e+04,4.964382e-01,4.070012e-01
min,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00
25%,3.350000e+02,3.220000e+02,1.809000e+03,0.000000e+00,0.000000e+00
50%,1.506000e+03,1.274000e+03,4.361000e+03,0.000000e+00,0.000000e+00
75%,4.746000e+03,1.085300e+04,1.234300e+04,1.000000e+00,0.000000e+00
max,7.621000e+04,2.512760e+05,1.661130e+05,1.000000e+00,1.000000e+00


In [19]:
df.count()

external_author_id    2946207
content               2946206
region                2937389
language              2946207
publish_date          2946207
harvested_date        2946207
following             2946207
followers             2946207
updates               2946207
post_type             1297582
account_type          2946207
retweet               2946207
account_category      2946207
new_june_2018         2946207
alt_external_id       2946207
tweet_id              2946207
article_url           2946207
tco1_step1            2100236
tco2_step1             711117
tco3_step1              15000
author                2946207
dtype: int64